In [ ]:
import pandas as pd 

df = pd.read_csv("llm.csv")

id_arXiv = df['id']

id_arXiv[:500]

In [ ]:
import requests
import json
import time

# Get all of the data for the fields 
# Put into an dictionary 
# Transform into a dataframe 

list_paper = []

num_limit = 500
num_loop = len(id_arXiv) // num_limit + 1

for i in range(num_loop):
    
    if i != 0:
        time.sleep(3)

        print("Wait for 3 seconds")
    
    if (i == num_loop - 1):
        
        ids_array = id_arXiv[500*i:len(id_arXiv)]
    else:
        ids_array = id_arXiv[500*i:500*(i+1)]
        
    
    ids_array = ids_array.apply(lambda x: f"ARXIV:{x}").tolist()
        

    fields = ','.join([
        'corpusId', 'url', 'title', 'venue', 'year', 'authors', 
        'referenceCount', 'citationCount', 'influentialCitationCount', 'isOpenAccess',
        'fieldsOfStudy', 's2FieldsOfStudy', 'externalIds'
    ])

    r = requests.post(
        'https://api.semanticscholar.org/graph/v1/paper/batch',
        params={'fields': fields},
        json={"ids": ids_array}
    )
    
   # Check if r.json() is not None
    

    list_paper.extend(r.json())

    print(i, len(list_paper))



In [ ]:
len(list_paper)

In [ ]:
id_arXiv

In [ ]:
import pandas as pd

# Assuming list_paper contains dictionaries
# If the structure of the dictionaries is consistent, you can directly convert it to a DataFrame
# df = pd.DataFrame(list_paper)

# If the structure is not consistent, you might need to normalize the data first
df_new = pd.json_normalize(list_paper)


In [ ]:
df_new = df_new.dropna(subset = ['paperId']).reset_index(drop = True)

df_new

In [ ]:
df_new.to_csv('semantic_scholar_with_id.csv',index = False)